In [ ]:
#-----------------------------------------
import pandas as pd
import numpy as np
import requests
#----------------------------------------- static plots analysis
import matplotlib.pyplot as plt
import seaborn as sns
#-----------------------------------------interactive plots
import plotly.express as px
import plotly.graph_objects as go
#----------------------------------------------------------------------
import datetime
#----------------------------------------------------------------------
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
#---------------------------------------------------------------------
from functools import reduce
#--------------------------------------------------------------------
import pytz
#--------------------------------------------------------------------
fig = go.Figure()
fig1 = go.Figure()
fig2 = go.Figure()
fig3 = go.Figure()
fig4 = go.Figure()
fig5 = go.Figure()
fig6 = go.Figure()
fig7 = go.Figure()
fig8 = go.Figure()
#--------------------------------------------------------------

##### Defined Functions

In [ ]:
AQ_G5185_url = ('https://thingspeak.com/channels/2159892/feeds.json?start=2023-07-20T00:00:00Z&end=2023-08-01T19:00:00Z&api_key=FGOVAJYAEGIMND15&timezone=Africa/Nairobi')
AQ_G5189_url = ('https://thingspeak.com/channels/2161673/feeds.json?start=2023-07-20T00:00:00Z&end=2023-08-01T19:00:00Z&api_key=C1T37N5RLHZ84CYN&timezone=Africa/Nairobi')
AQ_G5191_url = ('https://thingspeak.com/channels/2162918/feeds.json?start=2023-07-20T00:00:00Z&end=2023-08-01T19:00:00Z&api_key=IU0E93AWP1UL6U7T&&timezone=Africa/Nairobi')
test_device_url = ('https://thingspeak.com/channels/1014691/feeds.json?start=2023-07-20T00:00:00Z&end=2023-08-01T19:00:00Z&api_key=IU0E93AWP1UL6U7T&&timezone=Africa/Nairobi')

def collect_data(url):
   feeds_ = requests.get(url).json()['feeds']
   feeds_data = pd.DataFrame(feeds_)
   feeds_data.drop(columns = ['field2','field4','field5','field6','field8','entry_id'],inplace = True)
   feeds_data.rename(columns ={'field1': 'sensor1_PM2.5','field3':'sensor2_PM2.5','created_at':'timestamp','field7':"Battery Voltage"},inplace = True)
   feeds_data['timestamp'] = pd.to_datetime(feeds_data['timestamp'])
   feeds_data['hour'] = feeds_data['timestamp'].dt.hour
   feeds_data['day'] = feeds_data['timestamp'].dt.day
   feeds_data.set_index('timestamp',inplace = True)
   feeds_data = feeds_data.astype({'sensor1_PM2.5':'float','sensor2_PM2.5':'float','Battery Voltage':'float'}).round(2).reset_index()
   return feeds_data

def Battery_hourly(dataframe_):
  dataframe_ = dataframe_.groupby(['hour'])['Battery Voltage'].mean().reset_index()
  return dataframe_

def PM_hourly(dataframe_):
  dataframe_ = dataframe_.assign(average_PM2_5 = lambda x : (dataframe_['sensor1_PM2.5'] + dataframe_['sensor2_PM2.5'])/2)
  dataframe_ = dataframe_.groupby(['hour'])['average_PM2_5'].mean().reset_index()
  return dataframe_

def PM_Daily(dataframe_):
  dataframe_ = dataframe_.assign(average_PM2_5 = lambda x : (dataframe_['sensor1_PM2.5'] + dataframe_['sensor2_PM2.5'])/2)
  dataframe_.set_index('timestamp', inplace=True)
  dataframe_ = dataframe_.resample('D')['average_PM2_5'].mean().reset_index()
  return dataframe_

def PM_by_hour(dataframe_):
  dataframe_ = dataframe_.assign(average_PM2_5 = lambda x : (dataframe_['sensor1_PM2.5'] + dataframe_['sensor2_PM2.5'])/2)
  dataframe_.set_index('timestamp', inplace=True)
  dataframe_ = dataframe_.resample('H')['average_PM2_5'].mean().reset_index()
  return dataframe_

def uptime(dataframe_):
  dataframe_.set_index('timestamp', inplace=True)
  dataframe_ = dataframe_.resample('H')['Battery Voltage'].mean().reset_index()
  return dataframe_

def intra_sensor(dataframe_):
  #dataframe_ = dataframe_.resample('H')['sensor1_PM2.5','sensor2_PM2.5'].mean().reset_index()
  dataframe_ = dataframe_.groupby(['hour'])['sensor1_PM2.5','sensor2_PM2.5'].mean().reset_index()
  return dataframe_

def Intra_sensor_correlation(dataframe_):
  correlation = dataframe_['sensor1_PM2.5'].corr(dataframe_['sensor2_PM2.5'])
  return correlation

def Intra_sensor_variance(dataframe_):
  variance = dataframe_['sensor1_PM2.5'].var(dataframe_['sensor2_PM2.5'])
  return variance


def inter_sensor_correlation(dataframe_,dataframe_2,dataframe_3):
  dataframe_ = dataframe_.assign(average_PM2_5 = lambda x : (dataframe_['sensor1_PM2.5'] + dataframe_['sensor2_PM2.5'])/2).dropna()
  dataframe_2 = dataframe_2.assign(average_PM2_5 = lambda x : (dataframe_2['sensor1_PM2.5'] + dataframe_2['sensor2_PM2.5'])/2).dropna()
  dataframe_3 = dataframe_3.assign(average_PM2_5 = lambda x : (dataframe_3['sensor1_PM2.5'] + dataframe_3['sensor2_PM2.5'])/2).dropna()
  combined_df = pd.DataFrame()
  combined_df['100%_SPV'] = dataframe_['average_PM2_5']
  combined_df['75%_SPV'] = dataframe_2['average_PM2_5']
  combined_df['0%_SPV'] = dataframe_3['average_PM2_5']
  correlation =  combined_df[['100%_SPV','75%_SPV', '0%_SPV']].corr()
  return correlation

def inter_sensor_variance(dataframe_,dataframe_2,dataframe_3):
  dataframe_ = dataframe_.assign(average_PM2_5 = lambda x : (dataframe_['sensor1_PM2.5'] + dataframe_['sensor2_PM2.5'])/2).dropna()
  dataframe_2 = dataframe_2.assign(average_PM2_5 = lambda x : (dataframe_2['sensor1_PM2.5'] + dataframe_2['sensor2_PM2.5'])/2).dropna()
  dataframe_3 = dataframe_3.assign(average_PM2_5 = lambda x : (dataframe_3['sensor1_PM2.5'] + dataframe_3['sensor2_PM2.5'])/2).dropna()
  combined_df = pd.DataFrame()
  combined_df['100%_SPV'] = dataframe_['average_PM2_5']
  combined_df['75%_SPV'] = dataframe_2['average_PM2_5']
  combined_df['0%_SPV'] = dataframe_3['average_PM2_5']
  variance =  combined_df[['100%_SPV','75%_SPV', '0%_SPV']].var()
  return variance


def inter_sensor_correlation_2(dataframe_,dataframe_2,dataframe_3):
  combined_df = pd.DataFrame()
  combined_df['100%_SPV'] = dataframe_['average_PM2_5']
  combined_df['75%_SPV'] = dataframe_2['average_PM2_5']
  combined_df['0%_SPV'] = dataframe_3['average_PM2_5']
  correlation =  combined_df[['100%_SPV','75%_SPV', '0%_SPV']].corr()
  return correlation

def inter_sensor_variance_2(dataframe_,dataframe_2,dataframe_3):
  combined_df = pd.DataFrame()
  combined_df['100%_SPV'] = dataframe_['average_PM2_5']
  combined_df['75%_SPV'] = dataframe_2['average_PM2_5']
  combined_df['0%_SPV'] = dataframe_3['average_PM2_5']
  variance =  combined_df[['100%_SPV','75%_SPV', '0%_SPV']].var()
  return variance

##### Data Import and Functions

In [ ]:
# Get the data from thingspeak
D_100  = collect_data(AQ_G5185_url)
D_75 = collect_data(AQ_G5189_url)
D_0 = collect_data(AQ_G5191_url)
test_device = collect_data(test_device_url)
# Average Battery: Diunal
D_100_H_Battery = Battery_hourly(D_100)
D_75_H_Battery = Battery_hourly(D_75)
D_0_H_Battery = Battery_hourly(D_0)
test_Battery = Battery_hourly(test_device)
# Average PM2.5: Diunal
D_100_H_PM = PM_hourly(D_100)
D_75_H_PM = PM_hourly(D_75)
D_0_H_PM = PM_hourly(D_0)
test_H_PM = PM_hourly(test_device)
# daily average
daily_100 = PM_Daily(D_100)
daily_75 = PM_Daily(D_75)
daily_0 = PM_Daily(D_0)
test_daily = PM_Daily(test_device)
#By hour Average
houlry_100 = PM_by_hour(D_100)
houlry_75 = PM_by_hour(D_75)
houlry_0 = PM_by_hour(D_0)
test_houlry = PM_by_hour(test_device)
# Device on time evaluation
H_100 = uptime(D_100)
H_75 = uptime(D_75)
H_0 = uptime(D_0)
test_ = uptime(test_device)
# intra_sensors valuation
H_I_100 = intra_sensor(D_100)
H_I_75 = intra_sensor(D_75)
H_I_0 = intra_sensor(D_0)
test_intra = intra_sensor(test_device)
# intra sensors correlation
Intra_100 = Intra_sensor_correlation(D_100)
Intra_75  = Intra_sensor_correlation(D_75)
Intra_0 = Intra_sensor_correlation(D_0)
test_intra_1 = Intra_sensor_correlation(test_device)



<ipython-input-401-60245810f446>:46: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-401-60245810f446>:46: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-401-60245810f446>:46: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-401-60245810f446>:46: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
D_0.head(2)

,sensor1_PM2.5,sensor2_PM2.5,Battery Voltage,hour,day
timestamp,,,,,
2023-07-20 03:01:20+03:00,28.60,25.37,3.52,3,20
2023-07-20 03:02:28+03:00,28.43,27.83,3.52,3,20


In [ ]:
D_0_H_Battery.head(2)

,hour,Battery Voltage
0,0,3.665368
1,1,3.628158


In [ ]:
D_0_H_PM.head(2)

,hour,average_PM2_5
0,0,52.863175
1,1,44.075746


In [ ]:
daily_0.head(2)

,timestamp,average_PM2_5
0,2023-07-20 00:00:00+03:00,30.264385
1,2023-07-21 00:00:00+03:00,39.292959


In [ ]:
H_I_0.head(2)

,hour,sensor1_PM2.5,sensor2_PM2.5
0,0,54.232730,51.493620
1,1,45.134474,43.017018


#### Results plotting

In [ ]:
fig.add_trace(go.Scatter(x=D_100_H_Battery['hour'],y=D_100_H_Battery['Battery Voltage'],mode='lines+markers',name='100% SPV'))
fig.add_trace(go.Scatter(x=D_75_H_Battery['hour'],y=D_75_H_Battery['Battery Voltage'],mode='lines+markers',name='75% SPV'))
fig.add_trace(go.Scatter(x=D_0_H_Battery['hour'],y=D_0_H_Battery['Battery Voltage'],mode='lines+markers',name='0% SPV'))
fig.update_layout(title_text='Diurnal Battery analysis based on the period of study [07-20 : 08-01/2023]')
fig.update_xaxes(
    title='Hour',
    tickvals=[0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],  # Set the tick values (the points on the x-axis where the ticks will be displayed)
)
fig.update_yaxes(
    title='Mean Battery ')
fig.show()

In [ ]:
fig1.add_trace(go.Scatter(x=D_100_H_PM['hour'],y=D_100_H_PM['average_PM2_5'],mode='lines+markers',name='100% SPV'))
fig1.add_trace(go.Scatter(x=D_75_H_PM['hour'],y=D_75_H_PM['average_PM2_5'],mode='lines+markers',name='75% SPV'))
fig1.add_trace(go.Scatter(x=D_0_H_PM['hour'],y=D_0_H_PM['average_PM2_5'],mode='lines+markers',name='0% SPV'))
fig1.update_layout(title_text='Diurnal AveragePM analysis based on the period of study [07-20 : 08-01/2023]')
fig1.update_xaxes(
    title='Hour',
    tickvals=[0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],  # Set the tick values (the points on the x-axis where the ticks will be displayed)
)
fig1.update_yaxes(
    title='Mean PM ')
fig1.show()

In [ ]:
fig2.add_trace(go.Scatter(x=daily_100['timestamp'],y=daily_100['average_PM2_5'],mode='lines+markers',name='100%'))
fig2.add_trace(go.Scatter(x=daily_75['timestamp'],y=daily_75['average_PM2_5'],mode='lines+markers',name='75%'))
fig2.add_trace(go.Scatter(x=daily_0['timestamp'],y=daily_0['average_PM2_5'],mode='lines+markers',name='0%'))
fig2.update_layout(title_text='Daily analysis based on the period of study [10-23/07/2023]')
fig2.update_xaxes(
    title='Date'
)
fig2.update_yaxes(
    title='Mean PM2.5')
fig2.show()

In [ ]:
fig3.add_trace(go.Scatter(x=H_100['timestamp'],y=H_100['Battery Voltage'],mode='lines+markers',name='100%'))
fig3.add_trace(go.Scatter(x=H_75['timestamp'],y=H_75['Battery Voltage'],mode='lines+markers',name='75%'))
fig3.add_trace(go.Scatter(x=H_0['timestamp'],y=H_0['Battery Voltage'],mode='lines+markers',name='0%'))
fig3.update_layout(title_text='Hourly analysis of uptime based on the period of study [10-23/07/2023]')
fig3.update_xaxes(
    title='Date'
)
fig3.update_yaxes(
    title='Mean Battery')
fig3.show()
# Notable from the graph is: 0% spv: device goes off by 4am. 75% goes off by 5am while 100% doesnot go off on a good day

In [ ]:
fig4.add_trace(go.Scatter(x=H_I_100['hour'],y=H_I_100['sensor1_PM2.5'],mode='lines+markers',name='100%_sensor_one'))
fig4.add_trace(go.Scatter(x=H_I_100['hour'],y=H_I_100['sensor2_PM2.5'],mode='lines+markers',name='100%_sensor_tw0'))

fig4.add_trace(go.Scatter(x=H_I_75['hour'],y=H_I_75['sensor1_PM2.5'],mode='lines+markers',name='75%_sensor_one'))
fig4.add_trace(go.Scatter(x=H_I_75['hour'],y=H_I_75['sensor2_PM2.5'],mode='lines+markers',name='75%_sensor_tw0'))

fig4.add_trace(go.Scatter(x=H_I_0['hour'],y=H_I_0['sensor1_PM2.5'],mode='lines+markers',name='0%_sensor_one'))
fig4.add_trace(go.Scatter(x=H_I_0['hour'],y=H_I_0['sensor2_PM2.5'],mode='lines+markers',name='0%_sensor_tw0'))

fig4.update_layout(title_text='Hourly analysis of sensors based on the period of study [10-23/07/2023]')
fig4.update_xaxes(
    title='Hour',
    tickvals=[0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],  # Set the tick values (the points on the x-axis where the ticks will be displayed)
)
fig4.update_yaxes(
    title='Hourly Mean PM2.5')
fig4.show()

##### Non Graphical intepretations and analysis

In [ ]:
correlation_values = [['100%',Intra_100],['75%',Intra_75],['0%',Intra_0]]
sensor_correlation_hourly = pd.DataFrame(correlation_values,columns=['Device_corr','corr_val'])
sensor_correlation_hourly.set_index('Device_corr',inplace = True)
sensor_correlation_hourly

,corr_val
Device_corr,
100%,0.996301
75%,0.995924
0%,0.994589


In [ ]:
correlation_matrix_raw = inter_sensor_correlation(D_100,D_75,D_0)
correlation_matrix_raw

,100%_SPV,75%_SPV,0%_SPV
100%_SPV,1.000000,0.997961,0.994763
75%_SPV,0.997961,1.000000,NaN
0%_SPV,0.994763,NaN,1.000000


In [ ]:
correlation_matrix_hourly =inter_sensor_correlation_2(houlry_100,houlry_75,houlry_0)
correlation_matrix_hourly

,100%_SPV,75%_SPV,0%_SPV
100%_SPV,1.000000,0.971473,0.914787
75%_SPV,0.971473,1.000000,0.872390
0%_SPV,0.914787,0.872390,1.000000


In [ ]:
correlation_matrix_daily =inter_sensor_correlation_2(daily_100,daily_75,daily_0)
correlation_matrix_daily

,100%_SPV,75%_SPV,0%_SPV
100%_SPV,1.000000,0.966789,0.941203
75%_SPV,0.966789,1.000000,0.970960
0%_SPV,0.941203,0.970960,1.000000


In [ ]:
variance_matrix_raw = inter_sensor_variance(D_100,D_75,D_0)
variance_matrix_raw

100%_SPV    965.130526
75%_SPV     720.835794
0%_SPV      591.750289
dtype: float64

In [ ]:
variance_matrix_hourly =inter_sensor_variance_2(houlry_100,houlry_75,houlry_0)
variance_matrix_hourly

100%_SPV    916.622251
75%_SPV     816.326239
0%_SPV      372.010246
dtype: float64

In [ ]:
variance_matrix_daily =inter_sensor_variance_2(daily_100,daily_75,daily_0)
variance_matrix_daily

100%_SPV    144.020703
75%_SPV      90.378457
0%_SPV       60.029288
dtype: float64